In [12]:
import os 
import sys
import numpy as np
import matplotlib.pyplot as plt
import sklearn.gaussian_process as gpr
import math
import cmath
import seaborn as sns
from matplotlib.pyplot import cm

os.chdir('C:/Users/juelpb/Desktop/optimizeprogram')
path = os.getcwd()
sys.path.append(path)

In [13]:
d_path = path + '/Scripts/Optimize/SamplingData'

f_vcr = d_path + '/FlutterSpeeds_2022.csv'
f_cost          = d_path + '/ConfigCost_2022.csv'

vcr = np.loadtxt(f_vcr, delimiter=',')

row_idx = np.argwhere(vcr==3.5)

def pull_vcr_slice(tH_or_gH):
    row_idx = np.argwhere(vcr==tH_or_gH)[:,0]
    
    x_data = []
    y_data = []
    
    if tH_or_gH > 100:
        for row in row_idx:
            x_data.append(vcr[row][1])
            y_data.append(vcr[row][2])
        
    else:
        for row in row_idx:
            x_data.append(vcr[row][0])
            y_data.append(vcr[row][2])
    
    return x_data,y_data    
        

    


x_data, y_data = pull_vcr_slice(200)

plt.figure()
plt.plot(x_data,y_data)
plt.title('Slice @ tower height = 200 m')
plt.ylabel('$V_{cr}$')
plt.xticks([3.5,3.75,4.0,4.25,4.5])
plt.xlabel('Girder height [m]')
plt.show()

In [14]:
def Optimize(p_type, threshold, plot=True):
    # Create GPR for V_cr
    vcr_data = np.loadtxt(path + f'/Scripts/Optimize/SamplingData/FlutterSpeeds_{p_type}.csv', delimiter=',')
    cost_data = np.loadtxt(path + f'/Scripts/Optimize/SamplingData/ConfigCost_{p_type}.csv', delimiter=',')
    t_heights = vcr_data[:,0]
    g_heights = vcr_data[:,1]
    X = [[i,j] for i in t_heights for j in g_heights]
    X = np.array(X)
    V_cr = vcr_data[:,2]
    Cost = cost_data[:,2]
    Xs = []
    for i in range(len(t_heights)):
        Xs.append([t_heights[i], g_heights[i]]) 
    
    Xs = np.array(Xs)

    t_height_low = np.amin(t_heights)
    t_height_high = np.amax(t_heights)
    t_height_step = (t_height_high - t_height_low)/len(t_heights)
    
    g_height_low = np.amin(g_heights)
    g_height_high = np.amax(g_heights)
    g_height_step = (g_height_high - g_height_low)/len(g_heights)
    
    vcr_kernel = gpr.kernels.Matern([t_height_step, g_height_step], ([1, 10],[1, 10]), nu=2.5)
    vcr_model = gpr.GaussianProcessRegressor(kernel=vcr_kernel,
                                        alpha=1e-10)
                                        #normalize_y=True)
    cost_kernel = gpr.kernels.Matern([t_height_step, g_height_step], ([1, 10],[1, 10]), nu=2.5)
    cost_model = gpr.GaussianProcessRegressor(kernel=cost_kernel,
                                        alpha=1e-10)
    
    vcr_model.fit(Xs, V_cr)
    cost_model.fit(Xs, Cost)

    mesh = 50
    x0 = np.linspace(t_height_low, t_height_high, mesh)
    x1 = np.linspace(g_height_low, g_height_high, mesh)
    x0x1 = [[i,j] for i in x0 for j in x1]
    x0x1 = np.array(x0x1)
    x0,x1 = np.meshgrid(x0,x1)


    vcr_pred, std = vcr_model.predict(x0x1, return_std=True)
    cost_pred, std = cost_model.predict(x0x1, return_std=True)
    
    configs = []
    for i in range(len(vcr_pred)):
        if vcr_pred[i] > threshold:
            configs.append([x0x1[i,0], x0x1[i,1], cost_pred[i], vcr_pred[i]])
    
    configs_arr = np.array(configs)
    Optimal = configs_arr[np.argmin(configs_arr[:,2])]
    
    Z_vcr = np.reshape(vcr_pred,(mesh,mesh), order='F')
    Z_cost = np.reshape(cost_pred,(mesh,mesh), order='F')

    
    
    x = x1[:,0]
    y1 = Z_vcr[:,5]
    y2 = Z_vcr[:,25]
    y3 = Z_vcr[:,45]
    
    y1_min = np.argmin(y1)
    y1_max = np.argmax(y1)
    
    y2_min = np.argmin(y2)
    y2_max = np.argmax(y2)
    
    y3_min = np.argmin(y3)
    y3_max = np.argmax(y3)

        
    plt.figure(figsize=(8,3))
    
    plt.plot(x,y1,label='T=170',color='b')
    plt.scatter(x[y1_min],y1[y1_min],marker='x',color='b')
    plt.annotate(str(round(x[y1_min],3)), (x[y1_min],y1[y1_min]+0.5),color='b')
    plt.scatter(x[y1_max],y1[y1_max],marker='x',color='b')
    plt.annotate(str(round(x[y1_max],3)), (x[y1_max],y1[y1_max]+0.5),color='b')
    
    plt.plot(x,y2,label='T=200',color='r')
    plt.scatter(x[y2_min],y2[y2_min],marker='x',color='r')
    plt.annotate(str(round(x[y2_min],3)), (x[y2_min],y2[y2_min]+0.5),color='r')
    plt.scatter(x[y2_max],y2[y2_max],marker='x',color='r')
    plt.annotate(str(round(x[y2_max],3)), (x[y2_max],y2[y2_max]+0.5),color='r')
    
    plt.plot(x,y3,label='T=210',color='g')
    plt.scatter(x[y3_min],y3[y3_min],marker='x',color='g')
    plt.annotate(str(round(x[y3_min],3)), (x[y3_min],y3[y3_min]+0.5),color='g')
    plt.scatter(x[y3_max],y3[y3_max],marker='x',color='g')
    plt.annotate(str(round(x[y3_max],3)), (x[y3_max],y3[y3_max]-1.5),color='g')
    
    plt.title('Critical wind speed for tower height slices')
    plt.ylabel('$V_{cr}$')
    plt.xticks([3.5,3.75,4.0,4.25,4.5])
    plt.xlabel('Girder height [m]')
    plt.legend()
    plt.tight_layout()
    plt.savefig(path + '/Attachments/Optimize/VcrSlice.png',dpi=300)
    print(path + '/Attachments/Optimize/VcrSlice.png')
    
        
Optimize(2022, 76, plot=True)

c:\Users\juelpb\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarning: The optimal value found for dimension 1 of parameter length_scale is close to the specified lower bound 1. Decreasing the bound and calling fit again may find a better value.
c:\Users\juelpb\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\gaussian_process\kernels.py:430: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified upper bound 10. Increasing the bound and calling fit again may find a better value.


C:\Users\juelpb\Desktop\optimizeprogram/Attachments/Optimize/VcrSlice.png


In [15]:
def Optimize(p_type, threshold, plot=True):
    # Create GPR for V_cr
    vcr_data = np.loadtxt(path + f'/Scripts/Optimize/SamplingData/FlutterSpeeds_{p_type}.csv', delimiter=',')
    cost_data = np.loadtxt(path + f'/Scripts/Optimize/SamplingData/ConfigCost_{p_type}.csv', delimiter=',')
    t_heights = vcr_data[:,0]
    g_heights = vcr_data[:,1]
    X = [[i,j] for i in t_heights for j in g_heights]
    X = np.array(X)
    V_cr = vcr_data[:,2]
    Cost = cost_data[:,2]
    Xs = []
    for i in range(len(t_heights)):
        Xs.append([t_heights[i], g_heights[i]]) 
    
    Xs = np.array(Xs)

    t_height_low = np.amin(t_heights)
    t_height_high = np.amax(t_heights)
    t_height_step = (t_height_high - t_height_low)/len(t_heights)
    
    g_height_low = np.amin(g_heights)
    g_height_high = np.amax(g_heights)
    g_height_step = (g_height_high - g_height_low)/len(g_heights)
    
    vcr_kernel = gpr.kernels.Matern([t_height_step, g_height_step], ([1, 10],[1, 10]), nu=2.5)
    vcr_model = gpr.GaussianProcessRegressor(kernel=vcr_kernel,
                                        alpha=1e-10)
                                        #normalize_y=True)
    cost_kernel = gpr.kernels.Matern([t_height_step, g_height_step], ([1, 10],[1, 10]), nu=2.5)
    cost_model = gpr.GaussianProcessRegressor(kernel=cost_kernel,
                                        alpha=1e-10)
    
    vcr_model.fit(Xs, V_cr)
    cost_model.fit(Xs, Cost)

    mesh = 50
    x0 = np.linspace(t_height_low, t_height_high, mesh)
    x1 = np.linspace(g_height_low, g_height_high, mesh)
    x0x1 = [[i,j] for i in x0 for j in x1]
    x0x1 = np.array(x0x1)
    x0,x1 = np.meshgrid(x0,x1)


    vcr_pred, std = vcr_model.predict(x0x1, return_std=True)
    cost_pred, std = cost_model.predict(x0x1, return_std=True)
    
    configs = []
    for i in range(len(vcr_pred)):
        if vcr_pred[i] > threshold:
            configs.append([x0x1[i,0], x0x1[i,1], cost_pred[i], vcr_pred[i]])
    
    configs_arr = np.array(configs)
    Optimal = configs_arr[np.argmin(configs_arr[:,2])]
    
    Z_vcr = np.reshape(vcr_pred,(mesh,mesh), order='F')
    Z_cost = np.reshape(cost_pred,(mesh,mesh), order='F')

    
    
    x = x0[0,:]
    print(x)
    y1 = Z_cost[5,:]
    y2 = Z_cost[25,:]
    y3 = Z_cost[45,:]
    
    y1_min = np.argmin(y1)
    y1_max = np.argmax(y1)
    
    y2_min = np.argmin(y2)
    y2_max = np.argmax(y2)
    
    y3_min = np.argmin(y3)
    y3_max = np.argmax(y3)

        
    plt.figure(figsize=(8,3))
    
    plt.plot(x,y1,label='T=170',color='b')
    plt.scatter(x[y1_min],y1[y1_min],marker='x',color='b')
    plt.annotate(str(round(x[y1_min],3)), (x[y1_min],y1[y1_min]+0.5),color='b')
    plt.scatter(x[y1_max],y1[y1_max],marker='x',color='b')
    plt.annotate(str(round(x[y1_max],3)), (x[y1_max],y1[y1_max]+0.5),color='b')
    
    plt.plot(x,y2,label='T=200',color='r')
    plt.scatter(x[y2_min],y2[y2_min],marker='x',color='r')
    plt.annotate(str(round(x[y2_min],3)), (x[y2_min],y2[y2_min]+0.5),color='r')
    plt.scatter(x[y2_max],y2[y2_max],marker='x',color='r')
    plt.annotate(str(round(x[y2_max],3)), (x[y2_max],y2[y2_max]+0.5),color='r')
    
    plt.plot(x,y3,label='T=210',color='g')
    plt.scatter(x[y3_min],y3[y3_min],marker='x',color='g')
    plt.annotate(str(round(x[y3_min],3)), (x[y3_min],y3[y3_min]+0.5),color='g')
    plt.scatter(x[y3_max],y3[y3_max],marker='x',color='g')
    plt.annotate(str(round(x[y3_max],3)), (x[y3_max],y3[y3_max]-1.5),color='g')
    
    plt.title('Critical wind speed for tower height slices')
    plt.ylabel('$V_{cr}$')
    #plt.xticks([3.5,3.75,4.0,4.25,4.5])
    plt.xlabel('Girder height [m]')
    plt.legend()
    plt.tight_layout()
    #plt.savefig(path + '/Attachments/Optimize/VcrSlice.png',dpi=300)
    
    
       
        
Optimize(2022, 76, plot=True)

[180.         180.81632653 181.63265306 182.44897959 183.26530612
 184.08163265 184.89795918 185.71428571 186.53061224 187.34693878
 188.16326531 188.97959184 189.79591837 190.6122449  191.42857143
 192.24489796 193.06122449 193.87755102 194.69387755 195.51020408
 196.32653061 197.14285714 197.95918367 198.7755102  199.59183673
 200.40816327 201.2244898  202.04081633 202.85714286 203.67346939
 204.48979592 205.30612245 206.12244898 206.93877551 207.75510204
 208.57142857 209.3877551  210.20408163 211.02040816 211.83673469
 212.65306122 213.46938776 214.28571429 215.10204082 215.91836735
 216.73469388 217.55102041 218.36734694 219.18367347 220.        ]


c:\Users\juelpb\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarning: The optimal value found for dimension 1 of parameter length_scale is close to the specified lower bound 1. Decreasing the bound and calling fit again may find a better value.
c:\Users\juelpb\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\gaussian_process\kernels.py:430: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified upper bound 10. Increasing the bound and calling fit again may find a better value.


In [16]:
from Scripts.DeflectionPrediction.DefPred import Deflection
%matplotlib qt
def CableVolume(t_height, g_height, p_type, max_stress=500e6, z_cable_midspan=88.8, L_bridgedeck=10):
    # Creating cable vector
    x = np.array([-L_bridgedeck/2, 0, L_bridgedeck/2])
    z = np.array([t_height - z_cable_midspan, 0, t_height - z_cable_midspan])
    p = np.polyfit(x, z, 2)
    

    
    x = np.linspace(-L_bridgedeck/2, L_bridgedeck/2, L_bridgedeck)
    z_cable = np.zeros_like(x)
    for i in range(len(z_cable)):
        z_cable[i] = p[0]*x[i]**2 + p[1]*x[i] + p[2]
    
    
    
    plt.figure()
    #plt.plot(x)
    plt.plot(z_cable,'x')
    plt.show()
    
    
    
    # Summing segment lengths
    cl = 0
    for i in range(len(z_cable)-1):
        dx = x[i+1] - x[i]
        dz = z_cable[i+1] - z_cable[i]
        dl = np.sqrt(dx**2 + dz**2)
        cl += dl
       
    # Calculating demanded cable area for selected geometry
    cable_sf_max = Deflection(t_height, g_height, p_type, 'cable_sf_max')
    cable_area = cable_sf_max/max_stress
    
    cv = 2*cl*cable_area
    return cv, cl


CableVolume(220,4.0, 2022)

c:\Users\juelpb\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\interpolate\_fitpack_impl.py:977: RuntimeWarning: No more knots can be added because the number of B-spline
coefficients already exceeds the number of data points m.
Probable causes: either s or m too small. (fp>s)
	kx,ky=5,5 nx,ny=13,13 m=45 fp=1911937279.196462 s=0.000000


(141.62136342422826, 260.46988767488904)